In [ ]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

import warnings
import scipy
import scipy.linalg as sl
from matplotlib import cm

warnings.simplefilter("ignore", UserWarning)

sigma0 = np.array([[1, 0], [0, 1]])
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigmaz = np.array([[1, 0], [0, -1]])

bhz_parameters = {
    'topo'     : { 'A': 1.0, 'B': 1.00, 'D': 0.0, 'M': 0.2,  'del_z': 0.0},
    'topo2'     : { 'A': 1.0, 'B': 1.00, 'D': 0.0, 'M': 1.0,  'del_z': 0.0},
    'triv'     : { 'A': 1.0, 'B': 1.00, 'D': 0.0, 'M': -0.2,  'del_z': 0.0},
    'lead'     : { 'A': 3.0, 'B': 1.00, 'D': 0.0, 'M': 0.0,  'del_z': 0.0}
}


# Helper matrices for building hamiltonian
mx = np.kron(sigmax,0.5*(sigma0+sigmaz))
my = np.kron(sigmay,0.5*(sigma0+sigmaz))
mz1 = np.kron(sigmaz, 0.5 * (sigma0 + sigmaz))
mz2 = np.kron(sigmaz, 0.5 * (sigma0 - sigmaz))


# Onsite and hoppings for bhz model
def onsite(site, par):
    A, B, D, M = par.A/2, par.B, par.D, par.M
    return (M - 4 * B) * np.kron(sigma0, sigmaz) - 4 * D * np.kron(sigma0, sigma0) \
        + par.ez_x*mx + par.ez_y*my + par.eze_z*mz1 + par.ezh_z*mz2 \
        + par.del_z * np.kron(sigmay, sigmay)


def hopx(site1, site2, par):
    A, B, D, M = par.A/2, par.B, par.D, par.M
    return B * np.kron(sigma0, sigmaz) + D * np.kron(sigma0, sigma0) + 1j * A * np.kron(sigmaz, sigmax)


def hopy(site1, site2, par):
    A, B, D, M = par.A/2, par.B, par.D, par.M
    return B * np.kron(sigma0, sigmaz) + D * np.kron(sigma0, sigma0) - 1j * A * np.kron(sigma0, sigmay)
    
    
def two_terminal(L, w, par_lead):
    """ Make two terminal system with bhz model. """
    def shape(pos):
        (x, y) = pos
        return (0 <= y < w and 0 <= x < L)
    
    def lead_shape(pos):
        (x, y) = pos
        return (0 <= y < w)
    
    lat = kwant.lattice.square()
    sys = kwant.Builder()

    def hopping_x(site1, site2, par):
        xt, yt = site1.pos
        xs, ys = site2.pos
        return hopx(site1,site2,par) * np.exp(-0.5j * par.Bz * (xt - xs) * (yt + ys))
    
    # definition of system
    sys[lat.shape(shape, (0,0))] = lambda site, par: onsite(site,par) - par.mu*np.eye(4)
    sys[kwant.HoppingKind((1,0), lat)] = hopping_x
    sys[kwant.HoppingKind((0,1), lat)] = hopy
    
    # definition of leads
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0)))
    lead[lat.shape(lead_shape, (0,0))] = lambda site, par: onsite(site,par_lead) - par_lead.mu_lead*np.eye(4)
    lead[kwant.HoppingKind((1,0), lat)] = lambda site1, site2, par: hopping_x(site1, site2, par_lead)
    lead[kwant.HoppingKind((0,1), lat)] = lambda site1, site2, par: hopy(site1, site2, par_lead)
    
    # attaching leads
    sys.attach_lead(lead)
    sys.attach_lead(lead.reversed())
    
    return sys


def bhz_infinite(a=1):
    """ Make infinite system with bhz model. """
    def func(site, par):
        kx,ky = par.kx, par.ky
        H0, H1, H2 = onsite(site, par), hopx(site, site, par), hopy(site, site, par)
        
        return H0  \
            + H1*np.exp(-1j*kx) + H1.transpose().conjugate()*np.exp(+1j*kx) \
            + H2*np.exp(-1j*ky) + H2.transpose().conjugate()*np.exp(+1j*ky)
    
    lat = kwant.lattice.square()
    sys = kwant.Builder()
    sys[lat(0, 0)] = func

    return sys.finalized()


#Stuff for the dispersion plot:
def evaluate_on_grid(X, Y, func):
    """ X, Y should be in np.meshgrid form. It's enough for func to work on floats. """
    data = []
    for xx, yy in zip(X, Y):
        row = []
        for i,j in zip(xx, yy):
            row.append(func(i,j))
        data.append(row)
    data = np.array(data)
    data = [np.array(data[:,:,i]) for i in range(np.shape(data)[2])]
    return data


def dispersion_func(sys, par):
    def func(kx, ky):
        par.kx = kx
        par.ky = ky
        return diagonalize(sys, par)[0]
    return func


def diagonalize(sys, par):
    mat = sys.hamiltonian_submatrix(args=[par])
    ev, evec = scipy.linalg.eigh(mat)  # Automatically sorted
    return ev, evec


def plot_2D(X,Y,Z, ax_in=None):
    if ax_in==None:
        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = ax_in

    vmin = np.array(Z).min()
    vmax = np.array(Z).max()
    
    if len(np.shape(Z)) > 2:
        for z in Z:
            ax.plot_surface(X, Y, z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)
    else:
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)

    if ax_in==None:
        return fig, ax
    else:
        return ax 

##Topics

* Two limits: Mexican hat and weak pairing.
* Multiterminal conductance and length dependence
* Landau levels and a Landau level crossing
* Spin-orbit and localization of edge states by magnetic field

##Table of Contents
* [Introduction](#Introduction)
* [Two limits: Mexican hat and weak pairing](#Two-limits:-Mexican-hat-and-weak-pairing)
* [Quantized conductance and length dependence](#Quantized-conductance-and-length-dependence)
* [Landau levels](#Landau-levels)
* [Localization of the edge states by magnetic field](#Localization-of-the-edge-states-by-magnetic-field)
* [Summary](#Summary)


# Introduction

In [ ]:
MoocVideo("Gx-A0dWAjBg", src_location='5.2-intro')

This topic is special, since in order to meaningfully discuss experimental progress we need to do something we didn't do before in the course: we will show you the measurements and compare them with the *simple* theoretical expectations. Like this we will see what agrees and what doesn't.

All the figures showing the experiments are copyright Physical Society of Japan (2008), published in [
J. Phys. Soc. Jpn. 77, 031007 (2008)](http://journals.jps.jp/doi/abs/10.1143/JPSJ.77.031007) by  Markus König, Hartmut Buhmann, Laurens W. Molenkamp, Taylor Hughes, Chao-Xing Liu, Xiao-Liang Qi, and Shou-Cheng Zhang. They are available under CC-BY-NC-SA 4.0 International license.

# Two limits: Mexican hat and weak pairing

We just learned that topological insulators with inversion symmetry were simpler to think about. We will now use the  topological invariant to find a simple recipe for finding topological insulators. All we need to do is somehow vary the parity of the occupied states. One fact of nature that comes to our aid in this is that electrons in semiconductors typically occupy even parity $s$-orbitals and odd parity $p$-orbitals.

If we look up the bandstructure of a typical "non-topological" semiconductor, the highest valence-band is of odd parity and the lowest conduction band is even parity. As one moves down the periodic table to heavier elements with larger spin-orbit coupling the odd parity orbital switches spots with the even parity orbital. This *band inversion* is the domain where we can hope to find topological insulators.

Now you might think that all we have to do is go down the periodic table to heavier elements and just pick some material like HgTe (actually used for creation of QSHE), but that's not yet all. We still need to make a quantum well out of this semiconductor to make the system two-dimensional. This leads to two dimensional bands derived from the three dimensional band structure.

With a careful choice of widths one is able to get a band-inversion with a single odd parity band inverted with the even parity band. We saw from the last unit, that such a band-inversion changes to a topologically non-trivial value of the parity invariant. Right around the topological transition where the even and odd parity bands are degenerate, we can follow the discussion in the last unit to derive domain wall states at the edges.

We can write down the simplest Hamiltonian for an even and an odd parity band in a basis $|e,\sigma\rangle$ and $|o,\sigma\rangle$ on a block form 

$$H({\bf k})=\left(\begin{array}{cc}\epsilon_e({\bf k})&\Delta({\bf k})\\\Delta^\dagger({\bf k})&\epsilon_o({\bf k})\end{array}\right),$$

where $\Delta({\bf k})$ is the $2\times 2$ hybridization matrix. Inversion and time-reversal symmetries imply that $\Delta({\bf k})=-\Delta(-{\bf k})$ is odd under inversion and even under time-reversal.  Here we will focus on one such model $\Delta({\bf k})=\alpha\sigma_z(k_x+i k_y)$, which we call the Bernevig-Hughes-Zhang model. 

Since the even band is electron-like we approximate the even-band dispersion $\epsilon_e({\bf k})$
as $\epsilon_e({\bf k}) = \delta_e + m_e k^2$, while, for simplicity, we take the odd parity energy to be $\epsilon_o({\bf k})= \delta_o - m_o k^2$. The band inversion happens when $\delta_e < \delta_o$.

The spectrum of this Hamiltonian is very similar to that of a Chern insulator (after all we essentially just doubled the degrees of freedom). Just like in most of the topological systems, the shape of the band structure depends on the relative strength of band inversion versus inter-band coupling.

So below we see a qualitative band structure of one of the QSHE insulators, HgTe/CdTe quantum well, compared with the band structure of InAs/GaSb quantum well.

In [ ]:
def plot(par, ax):

    sys = bhz_infinite()

    K = np.linspace(-np.pi, np.pi, 75)
    mesh = np.meshgrid(K, K)
    energies = evaluate_on_grid(*mesh, func=dispersion_func(sys, par))

    plot_2D(*mesh, Z=energies, ax_in=ax)

    ax.set_xlabel('$k_x$')
    ax.set_xticks([-np.pi, 0.0, np.pi])
    ax.set_xticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_ylabel('$k_y$')
    ax.set_yticks([-np.pi, 0.0, np.pi])
    ax.set_yticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_zlabel('$E$')
    
    
    evals = list(range(-8,12,4))
    ax.set_zticks(evals)
    ax.set_zticklabels(["${0}$".format(i) for i in evals])
    

    ax.view_init(8,20)
    ax.set_zlim3d(-8,10)

#     plt.suptitle('$t = %1.1f,$ $ \mu = %1.1f,$ $ \Delta = %1.1f,$ $ \gamma = %1.1f$' %(par.t, par.mu, par.delta, par.gamma));

fig = plt.figure(figsize=[9.5, 4], tight_layout=True)
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')

plot_par = {
    'topo'     : { 'A': 0.5, 'B': 1.00, 'D': -0.1, 'M': 1.5,  'del_z': 0.},
    'triv'     : { 'A': 0.5, 'B': 1.00, 'D': -0.1, 'M': -0.2,  'del_z': 0.},
}

par1 = SimpleNamespace(mu=0, ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=0.0, **plot_par['triv'])
par2 = SimpleNamespace(mu=0, ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=0.0, **plot_par['topo'])

plot(par1, ax1);
plot(par2, ax2)

From the last unit, we understood the nature of the edge modes near the topological phase transition, where a doubled Dirac model was appropriate. Deep in the strongly band-inverted topological regime, the bulk band structure has a mexican hat structure with the gap proportional to $\alpha$. 

The edge modes in this regime are quite different in structure from those near the topological transition. To see this, let us first set $k_y=0$ in the Hamiltonian. If we set $\alpha=0$ then there are two fermi points where the dispersion is roughly linear - let us label these points by $\tau_z=\pm 1$. We can describe the edge of the system, by assigning boundary conditions to the $k_x=\pm k_F$ modes in terms of time-reversal invariant phase-shifts. 

The bulk solutions near $k_x\sim\pm k_F$ can be written as $\psi_\pm(x)=e^{-x/\xi}\psi_\pm(0)$. Matching boundary conditions, we find that a zero energy pair of edge solutions exists in the case of inverted bands. These solutions differ from the ones in the Dirac limit by the appearance of the oscillating part of the wave function.

# Quantized conductance and length dependence

Unlike in the case of Majoranas, not much thinking is required to figure out the relevant signature of quantum spin Hall effect. Since there is a pair of modes on each edge of the sample that is protected from back-scattering, and all the other modes are gapped or backscattered, then the edge states are the only ones to carry current, and this current will not suffer from back-scattering.

If we consider the simplest case, so a sample with only two terminals, then Landauer's formula together with the absence of back-scattering gives the conductance of $G_0=2 e^2/h$.

When we move the Fermi level outside of the bulk gap, the bulk becomes conducting and so the conductance increases.

We end up with this situation:

In [ ]:
par_lead = SimpleNamespace(ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=+1.5, **bhz_parameters['lead'])

sys = two_terminal(40, 40, par_lead=par_lead).finalized()

par_topo = SimpleNamespace(ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=+1.5, **bhz_parameters['topo'])
par_triv = SimpleNamespace(ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=+1.5, **bhz_parameters['triv'])

mu_max = .8
mus = np.linspace(-mu_max, mu_max, 50)
Gtopo = []
Gtriv = []


for mu in mus:    
    par_topo.mu = mu
    par_triv.mu = mu
    
    smatrix = kwant.smatrix(sys, energy=0.0, args=[par_topo])
    Gtopo.append(smatrix.transmission(1, 0))
    
    smatrix = kwant.smatrix(sys, energy=0.0, args=[par_triv])
    Gtriv.append(smatrix.transmission(1, 0))


def plot(mu):
    fig, ax = plt.subplots(1,3, figsize=([9.5, 4]), tight_layout=True)
    ax1, ax2, ax3 = ax

    ax2.set_color_cycle(['k'])
    ax3.set_color_cycle(['k'])

    ax1.plot(mus, Gtopo, 'r-')
    ax1.plot(mus, Gtriv, 'b-')
    gmax = 10
    ax1.plot([mu, mu], [0, gmax], 'b--')

    par = SimpleNamespace(mu=0.0, ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=0.0, **bhz_parameters['topo'])
    sys = two_terminal(50,20, par_lead=par).finalized()
    kwant.plotter.bands(sys.leads[0], args=[par], momenta=np.linspace(-np.pi/3, np.pi/3, 51), show=False, ax=ax2)

    par = SimpleNamespace(mu=0.0, ez_x=0.0, ez_y=0.0, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=0.0, **bhz_parameters['triv'])
    sys = two_terminal(50,20, par_lead=par).finalized()
    kwant.plotter.bands(sys.leads[0], args=[par], momenta=np.linspace(-np.pi/3, np.pi/3, 51), show=False, ax=ax3)

    ax1.set_xlabel(r'$\mu$')
    ax1.set_ylabel(r'$G\,[e^2/h]$')

    ax2.set_xlabel('$k$')
    ax2.set_ylabel('$E$')

    ax1.set_title('conductance')
    ax2.set_title('spectrum (topo)')
    ax3.set_title('spectrum (trivial)')

    ax2.plot([-4, 4], [mu, mu], 'k--')
    ax3.plot([-4, 4], [mu, mu], 'k--')

    ax1.set_xlim(-mu_max, mu_max)
    ax1.set_ylim(0, gmax)

    vals = np.arange(-.8, .9, 0.4)
    ax1.set_xticks(vals)
    ax1.set_xticklabels(["${0}$".format(i) for i in vals])

    vals = list(range(0, gmax+1, 2))
    ax1.set_yticks(vals)
    ax1.set_yticklabels(["${0}$".format(i) for i in vals])

    vals = np.arange(-2, 2.5, .5)
    ax2.set_yticks(vals)
    ax2.set_yticklabels(["${0}$".format(i) for i in vals])

    ax2.set_xlim(-np.pi/3,np.pi/3);
    ax2.set_ylim(-1.5, 1.5)
    
    ax2.set_xticks([-np.pi/3, 0.0, np.pi/3])
    ax2.set_xticklabels([r'$-\frac{\pi}{3}$', r'$0$', r'$\frac{\pi}{3}$'])
    
    vals = np.arange(-2, 2.5, .5)
    ax3.set_yticks(vals)
    ax3.set_yticklabels(["${0}$".format(i) for i in vals])

    ax3.set_xticks([-np.pi/3, 0.0, np.pi/3])
    ax3.set_xticklabels([r'$-\frac{\pi}{3}$', r'$0$', r'$\frac{\pi}{3}$'])

    ax3.set_xlim(-np.pi/3,np.pi/3);
    ax3.set_ylim(-1.5, 1.5)

    return fig

StaticInteract(lambda mu: plot(.2*mu), mu = RangeWidget(-4, 4, name='mu', default=0, show_range=True))

Here on the left we see comparison between conductances of a trivial (blue curve) and a topological insulator (red curve) as a function of chemical potential. The other two panels show spectra of the QSHE and a trivial insulator in the scattering region. As we expected, conductance is quantized when the chemical potential is inside the band gap of a QSHE system

Let's now see what can be measured experimentally.

![](figures/konig_fig13.png)

(copyright JPS, see license in the beginning of the chapter)

What you notice is that the maximum resistance for the 4.5 nm thick quantum well is much higher than the 8nm thick well. Given that theory predicts that HgTe quantum wells described by Michael Wimmer in his video are topological when their thickness is between 6.3 nm and 12 nm, this measurement is suggestive that the lower resistance of the 8nm thick well might be as a result of edge conductance. At this point, you might complain that the resistance for the 8nm well, even though lower of the two, looks more like $100k\Omega$ rather than the predicted $12k\Omega$ from the quantum of conductance.

The black curve here is the resistance of a trivial insulator, and the red one should be that of topological one. The resistance of a trivial insulator becomes very high as we would expect, and there is a plateau-like feature in a topological regime.

Fortunately, further experiments revealed that when the Wurzburg group reduced the length of the sample from length $L=20\mu m$ to $L=1\mu m$ as seen below the conductance maximum rises to about $12.9k\Omega$:

![](figures/konig_fig14.png)

(copyright JPS, see license in the beginning of the chapter)

We see something different from what we expected: the average resistance value at the plateau is correct, but only within 10% precision, very different from the $10^{-8}$ accuracy of the quantum Hall effect.

This difference most likely originated from back-scattering due to some source. In quantum Hall, back-scattering is prohibited by the absence of modes going in the other direction, while now the protection is much weaker and is merely due to the Kramers theorem.

The exact origin of the back-scattering is hard to understand. It could be inelastic scattering that does not preserve energy and thus avoids Kramers theorem, it could also be some residual magnetic impurities. One of the papers that we suggest for review proposes an interesting theory of the origin of the back-scattering, while another reports measurements of InAs/GaSb quantum well, where conductance seems much better quantized.

Regardless of the exact origin of back-scattering, at any finite temperature, there is an inelastic scattering length $l_\phi$ beyond which we do not expect any protection from scattering. When the edge length $l$ is larger than $l_\phi$, we expect the edge to turn into an incoherent conductor with resistance of $(e^2/h) l_\phi/l$. 

This allows us to in principle, even measure $l_\phi$ on the quantum spin hall edges by looking at the length dependence of the conductance. As we expect, the experiments find that small samples have the conductance close to $G_0$, while in large samples the conductance is suppressed.

# Landau levels

We learned that the key ingredient to obtain an inversion symmetric topological insulator is band inversion - an electron-like bands with positive effective mass and a hole-like band with negative effective mass invert in energy.

The standard way to distinguish electrons from holes is to measure the sign of the Hall resistance, which is positive for electrons and negative for holes. Hence, we expect to measure a change in the sign of the Hall conductance as we change the position of the Fermi level from being above to being below the band gap.

In the first plot below, you see traces of the Hall resistance of a quantum spin Hall sample as a function of the applied magnetic field, for several values of the gate voltage, given by different colors. You see that for $V_g=-1$ V the Hall resistance is positive, for $V_g = -2 V$ the resistance is negative. These are the two black traces. They both exhibit a very well formed $\nu=1$ quantum Hall plateau for high enough fields, and a vanishing Hall resistance for zero field. This is standard, expected behavior.

For some traces between these two values, the resistance shoots up to very high values. You have insulating behavior, because the Fermi level is in the middle of the band gap. This is also expected behavior.

However, you may notice something interesting. Let's focus for instance on the green and red traces taken for two very close values of $V_g$. Because these correspond to Fermi levels in the middle of the band gap, they show a very high resistance, except for a range of magnetic field values, where they also exhibit a quantum Hall plateau!

This proves what we hoped to find: there is a Landau of electrons that crosses with a Landau level of holes.

![](figures/konig_fig11.png)

(copyright JPS, see license in the beginning of the chapter)

As shown in the lower panel, this particular feature is due to the unique structure of Landau levels which you obtain in the presence of a band inversion.

The Landau levels of a electron-like band have a positive slope as a function of magnetic field, while those of a hole-like band have a negative slope. In a trivial bandstructure, you would have that all negative energy levels bend down as a function of field, while all positive energy levels bend up. As a consequence, if you place the Fermi level in the middle of the band gap and increase the magnetic field, no Landau level will ever cross the Fermi level.

However, in the presence of a band inversion, you obtain what is shown in the figure: the lowest Landau levels coming from the inverted bands go in the “wrong” direction. For a value of the magnetic field, you must have a crossing between them. Furthemore, they will both cross the Fermi level if that is placed in the middle of the zero-field band gap.

Due to this fact, in a certain range of fields you are able to observe a Hall effect even when you place the Fermi level in the middle of the zero-field band gap. And indeed, by comparing the experimental results with expected behavior of the Landau levels, you see that the positions of the Fermi-level crossings coincide with the re-entrant Hall plateaus of the experimental traces - as marked by the green and red arrows.

As a further confirmation that this effect is due to band inversion, this behavior was only observed in samples with a thickness above the threshold value expected to obtain a quantum spin Hall phase, and never in samples with a smaller thickness.

# Localization of the edge states by magnetic field

Theoretically, the hallmark of the topological insulator is the quantized conductance of the edge states that are protected from elastic back-scattering. From the last unit, we learned that the key to this protection is time-reversal symmetry. Therefore, we expect that the quantized conductance should be suppressed when we break time-reversal symmetry say by applying a magnetic field.

We can think about this more explicitly by considering a simple model for the helical edge states with a magnetic field $\bf B$:

$$H=v_F k_x\sigma_z+{\bf B}\cdot {\bf \sigma},$$

where $\bf \sigma$ are Pauli matrices representing the spin degree of freedom at the edge. This is what we would get from the BHZ model, which conserves spin. For more general models we would interpret $\bf \sigma$ as a pseudo-spin degree of freedom, which is odd under time-reversal.

If we consider the simple case of the magnetic field ${\bf B}=B {\bf x}$ along the x-direction, we would find that the edge spectrum becomes gapped with a spectrum written as $E=\pm\sqrt{v_F^2 k_x^2+B^2}$. Clearly, the edge becomes insulating for the chemical potential set at $E=0$.

We can calculate see that this is the case very easily if we plot conductance of the QSHE model as a function of magnetic field:

In [ ]:
par = SimpleNamespace(mu=0, ez_x=0.0, ez_y=0.4, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=1.5, **bhz_parameters['topo2'])
sys = two_terminal(40,20, par_lead=par).finalized()

field = []
data = []
for par.ez_y in np.linspace(0,0.15):

    # compute the scattering matrix at a given energy
    smatrix = kwant.smatrix(sys, energy=0.0, args=[par])

    # compute the transmission probability from lead 0 to
    # lead 1
    field.append(par.ez_y)
    data.append(smatrix.transmission(1, 0))
    
def plot(ez_y):
    fig = plt.figure(figsize=[9.5, 4], tight_layout=True)
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)


    ax1.plot(field, data)
    ax1.plot([ez_y, ez_y], [0, 2], 'b--')
    ax2.set_color_cycle(['k'])

    par = SimpleNamespace(mu=0, ez_x=0.0, ez_y=ez_y, eze_z=0.0, ezh_z=0.0, Bz=0, mu_lead=0.0, **bhz_parameters['topo2'])
    sys = two_terminal(40,20, par_lead=par).finalized()
    kwant.plotter.bands(sys.leads[0], args=[par], momenta=np.linspace(-np.pi/3, np.pi/3, 25), 
                        show=False, ax=ax2);

    ax1.set_ylim(-2,2)
    ax2.set_ylim(-2,2)

    ax1.set_xlabel('$E_Z$')
    ax1.set_ylabel(r'$G\,[e^2/h]$')

    ax2.set_xlabel('$k$')
    ax2.set_ylabel('$E$')

    ax1.set_title('conductance')
    ax2.set_title('band structure')

    ax2.plot([-4, 4], [0, 0], 'k--')

    evals = [0.05, .1, .15]
    ax1.set_xticks(evals)
    ax1.set_xticklabels(["${0}$".format(i) for i in evals])

    evals = [0, .5, 1, 1.5, 2.]
    ax1.set_yticks(evals)
    ax1.set_yticklabels(["${0}$".format(i) for i in evals])

    evals = np.arange(-2,2.5,.5)
    ax2.set_yticks(evals)
    ax2.set_yticklabels(["${0}$".format(i) for i in evals])

    ax2.set_xticks([-np.pi/3, 0.0, np.pi/3])
    ax2.set_xticklabels([r'$-\frac{\pi}{3}$', r'$0$', r'$\frac{\pi}{3}$'])

    ax1.set_ylim(0,2)
    ax2.set_ylim(-1.5,1.5)

    ax1.set_xlim(0,0.15)
    ax2.set_xlim(-np.pi/3,np.pi/3);
    
    return fig

StaticInteract(plot, ez_y = RangeWidget(0, .2, 0.05, name='Ez', default=0.05, show_range=True))

However, even if we consider energies $E>B$ above the gap, by calculating the eigenstates, you can easily check that the eigenstates at $\pm k_x$ are no longer Kramers' pairs that are related by time-reversal symmetry. Therefore, any mechanism which changes momentum by $2 k_x$ can scatter back electrons from left movers to right movers. 

Edges of semiconductors are typically quite disordered - so we expect the random potential at the edge to provide "elastic backscattering" that can change the momentum without changing the energy. Such backscattering, in addition to any other "inelastic backscattering" by phonons etc, would decrease the conductance of the edge from the ideal quantized value. If we set $B=0$, elastic back-scattering that can occur at finite $B$ is forbidden so we generally expect the application of a magnetic field to reduce conductance of the edge.

As we see below that indeed the conductance of the $L=20\,\mu m$ device is strongly reduced by the application of a magnetic field: 

![](figures/konig_fig15.png)

(copyright JPS, see license in the beginning of the chapter)

However, we notice that this effect seems to work only when the magnetic field is perpendicular to the sample. In-plane magnetic fields do not seem to do a whole lot (there is an effect, but much larger fields are required). According to our model Hamiltonian, an in-plane field should have opened a gap, while a perpendicular field which adds a term proportional to $\sigma_z$ should have not done anything.

So, while the experiment sees something similar to what we hoped to find using a simple theory, the effect of the magnetic field seems reversed. There may be several explanations for this phenomenon, such as a presence of extra terms in the Hamiltonian that rotate the spins of the edge states without breaking time-reversal symmetry.

You might be worried that the suppression of conductance is only shown for the long device, which does not show quantized conductance. If you are, then you are absolutely right in worrying about this :-)

Localization of QSHE edge states by magnetic field is relatively poorly understood, and we are not aware of a final experiment that would prove its existence or tell us in details what it is that happens at the QSHE edge in magnetic field. As you will learn in two weeks, opening the gap by magnetic field opens new pathways for creation of Majoranas, and so it is still a very important direction of research.

In [ ]:
question = ("Why did we not see a similar suppression of conductance with magnetic field in the case of  "
            "the quantum Hall effect in week 3?")

answers = ["The quantum Hall effect appeared in much higher quality samples.",
           "There was no spin in the quantum Hall effect, so the magnetic field could not couple to anything.",
           "The topological protection of chiral quantum Hall edges does not rely on time-reversal in the way "
           "the helical edges do.",
           "The suppression here arises from inelastic scattering, which could not arise in the quantum Hall case."]

explanation = ("The magnetic field dependence here arises from the fact that B breaks time-reversal symmetry, which is  "
               "required for the protection of the helical edge states. ")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=2, explanation=explanation)

# Summary

In [ ]:
MoocVideo("7M1vI9PhM6I", src_location='5.2-summary')

**Questions about what you just learned? Ask them below!**

In [ ]:
MoocDiscussion("Questions", "QSHE - experiments")